In [50]:
import TTest as main
import Picture.calculateW as calc
from keras.datasets import mnist
import Picture.convertToData as convert
import Picture.pictureToPixel as toPix
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical

data = main.ReadData(58000)

def GetRealY(index):
    list = []
    for i in data[1]:
        list.append(i[index])
    list = np.array(list)
    return list


x = np.array(data[0])
z = calc.GetYProd(np.array(data[0]), np.array([1] * len(data[0][0])), 1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [51]:
(trainImage, trainLabel), (testImage, testLabel) = mnist.load_data()
def GradientNew(x, y, w, b):
  yCalc = calc.GetYProd(x, w, b)
  bG = (yCalc - y).mean()
  wG = np.zeros(x.shape[1])
  for i in range(x.shape[1]):
    wG[i] = (x[:, i] * (yCalc - y)).mean()
  return wG, bG

def GradientRunNew(x, maxNum, learningRateNum, col):
  learningRate = 1
  y = GetRealY(col)
  initW = np.array([0.0] * len(data[0][0]))
  initB = 0
  costBack = -1
  for i in range(maxNum):
    stepW, stepB = GradientNew(x, y, np.array([1] * len(data[0][0])), initB)
    initW -= learningRate * stepW
    initB -= learningRate * stepB
    yCalc = calc.GetYProd(x, initW, initB)
    cost = calc.CalcDistance(y, yCalc)
    if(i % 100 == 0):
      print(f"Running, {i/maxNum * 100:.2f}%, cost: {cost}")
    if(costBack < 0 or costBack > cost):
      costBack = cost
    elif(costBack < cost):
      print("reverse")
      print(f"Running, {i/maxNum * 100:.2f}%, cost: {cost}")
      initW += learningRate * stepW
      initB += learningRate * stepB
      learningRate /= 10
      learningRateNum -= 1
      if(learningRateNum == 0):
        print(cost)
        print("over")
        return initW , initB
    elif (costBack == cost):
      print(cost)
      print("Equal to over")
      return initW, initB 

  return initW, initB
def GetCurrentImage(num, index):
    # (trainImage, trainLabel), (testImage, testLabel) = mnist.load_data()
    testLabels = to_categorical(trainLabel)
    pixel = trainImage[num]
    pixel = convert.PixelNormalize(pixel)
    pixel = convert.ToPooling([pixel, pixel, pixel], 28)
    line = np.array(convert.ToLine(pixel[0]))
    return pixel, line, testLabels[num][index]

def CheckAcc(line, ans, w, b):
    x = (line * w).sum() + b
    return x 

In [52]:
w = []
b = []
for i in range(10):
    print(f"run {i}")
    w1, b1 = GradientRunNew(x, 10000, 20, i)
    w.append(np.array(w1))
    b.append(np.array(b1))
w = np.array(w)
wNew = w.mean(axis=0)
b = np.array(b)
bNew = b.mean()

run 0
Running, 0.00%, cost: 0.09877585962015567
reverse
Running, 0.01%, cost: 0.0987758620689526
reverse
Running, 0.02%, cost: 0.09877586136577664
reverse
Running, 0.03%, cost: 0.09877585990898027
reverse
Running, 0.04%, cost: 0.09877585965071607
reverse
Running, 0.05%, cost: 0.09877585962322921
reverse
Running, 0.06%, cost: 0.09877585962046319
reverse
Running, 0.07%, cost: 0.09877585962018642
reverse
Running, 0.08%, cost: 0.09877585962015875
reverse
Running, 0.09%, cost: 0.09877585962015599
reverse
Running, 0.10%, cost: 0.09877585962015571
0.09877585962015567
Equal to over
run 1
Running, 0.00%, cost: 0.11255153148842559
reverse
Running, 0.01%, cost: 0.11255172412096731
reverse
Running, 0.02%, cost: 0.11255165471544083
reverse
Running, 0.03%, cost: 0.1125515504302014
reverse
Running, 0.04%, cost: 0.11255153347493094
reverse
Running, 0.05%, cost: 0.11255153168803338
reverse
Running, 0.06%, cost: 0.11255153150839597
reverse
Running, 0.07%, cost: 0.11255153149042271
reverse
Running, 0.08%

In [ ]:
import numpy as np


def Softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # 防止数值溢出
    softmax_probs = exp_logits / np.sum(exp_logits, axis=0)
    rounded_probs = np.round(softmax_probs * 100, decimals=2)
    return rounded_probs

In [58]:



(trainImage, trainLabel), (testImage, testLabel) = mnist.load_data()
trainLabel
numTotal = 0
numCorrect = 0
for i in range(58000, 60000):
    list = []
    for j in range(10):
        pixel, line, ans = GetCurrentImage(i, j)
        per = (int)(CheckAcc(line, ans, w[j], b[j])*100)/100
        list.append(per)
    sorted_indices = np.argsort(list)
    if(trainLabel[i] == sorted_indices[-1]):
        numCorrect += 1
    numTotal += 1
    print(f"Num of: {numTotal}, Answer is {trainLabel[i]} result is {sorted_indices[-1]}, accuracy: {numCorrect / numTotal * 100:.2f}%")

print(f"Final accuracy is {numCorrect / numTotal * 100:.2f}%")

Num of: 1, Answer is 2 result is 8, accuracy: 0.00%
Num of: 2, Answer is 3 result is 3, accuracy: 50.00%
Num of: 3, Answer is 0 result is 0, accuracy: 66.67%
Num of: 4, Answer is 4 result is 9, accuracy: 50.00%
Num of: 5, Answer is 9 result is 9, accuracy: 60.00%
Num of: 6, Answer is 5 result is 8, accuracy: 50.00%
Num of: 7, Answer is 9 result is 8, accuracy: 42.86%
Num of: 8, Answer is 6 result is 6, accuracy: 50.00%
Num of: 9, Answer is 1 result is 1, accuracy: 55.56%
Num of: 10, Answer is 7 result is 7, accuracy: 60.00%
Num of: 11, Answer is 6 result is 6, accuracy: 63.64%
Num of: 12, Answer is 8 result is 8, accuracy: 66.67%
Num of: 13, Answer is 0 result is 0, accuracy: 69.23%
Num of: 14, Answer is 9 result is 8, accuracy: 64.29%
Num of: 15, Answer is 0 result is 0, accuracy: 66.67%
Num of: 16, Answer is 0 result is 0, accuracy: 68.75%
Num of: 17, Answer is 6 result is 6, accuracy: 70.59%
Num of: 18, Answer is 1 result is 1, accuracy: 72.22%
Num of: 19, Answer is 4 result is 4, a

In [57]:
def GetLine(url):
    img = toPix.GetPixels(url)
    rgb = convert.SplitToRGB(img)
    rgb = convert.ToPooling(rgb, 28)
    line = convert.ToLine(rgb[0])
    return line

def GetName(url):
    line = GetLine(url)
    per = []
    for i in range(10):
        p = CheckAcc(line, "0", w[i], b[i])
        per.append(p)
    soft = Softmax(per)
    sorted_indices = np.argsort(per)
    return soft, sorted_indices[-1]

for i in range(10):
    a = GetName(f"./{i}.png")
    print(i, a[0], "->" ,a[1])

0 [77.15  0.17  8.11  2.97  1.06  1.1   6.08  0.81  2.07  0.48] -> 0
1 [ 4.59 21.66  9.73 15.18  3.83  4.75  8.25  7.6  15.81  8.6 ] -> 1
2 [24.12  0.64 15.5  17.44  3.92  2.98  3.72  9.18 17.08  5.42] -> 0
3 [ 7.51  0.93 19.09 34.31  4.32  2.44  2.98  6.04 17.19  5.19] -> 3
4 [14.11  0.88  9.71 11.6  18.41  2.41  8.59  7.13 10.06 17.1 ] -> 4
5 [19.79  0.9   6.8  27.17  4.76  6.47  5.27  2.61 19.94  6.29] -> 3
6 [23.62  0.99  8.1  18.03  2.8   4.41 18.4   2.26 16.42  4.97] -> 0
7 [11.24  1.46 22.8  18.63  6.54  3.55  7.11  8.29 13.49  6.88] -> 2
8 [10.27  1.28 21.14 15.55  3.25  2.54  8.17  6.1  27.23  4.47] -> 8
9 [16.53  0.64  6.53 31.52  4.92  3.91  3.68 10.97 15.52  5.77] -> 3
